Import Libraries

In [1]:
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest

Download Libraries For To Refine The Words

In [17]:
# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

True

Generic Function To Extract Text From PDF

In [18]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

Preprocess The Data

In [19]:
def preprocess_text(text):
    sentences = sent_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    processed_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence.lower())
        words = [lemmatizer.lemmatize(word) for word in words if word.isalnum()]
        words = [word for word in words if word not in stop_words]
        processed_sentences.append(' '.join(words))
    
    return processed_sentences, sentences

Summarize Data

In [25]:
def summarize_text(text, num_sentences=3):
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    word_frequencies = FreqDist(word for word in words if word not in stop_words)
    
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies:
                if len(sentence.split()) < 30:
                    if sentence not in sentence_scores:
                        sentence_scores[sentence] = word_frequencies[word]
                    else:
                        sentence_scores[sentence] += word_frequencies[word]
    
    summary_sentences = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary

Function to get an answer

In [26]:
def get_answer(question, context, top_n=5):
    processed_sentences, original_sentences = preprocess_text(context)
    processed_question = ' '.join(preprocess_text(question)[0])
    
    all_text = processed_sentences + [processed_question]
    
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_text)
    
    question_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(question_vector, tfidf_matrix[:-1])
    
    top_indices = similarities.argsort()[0][-top_n:][::-1]
    
    relevant_sentences = [original_sentences[i] for i in top_indices]
    
    # Combine the most relevant sentences
    combined_answer = ' '.join(relevant_sentences)
    
    # Summarize the combined answer to make it more coherent
    summarized_answer = summarize_text(combined_answer, num_sentences=2)
    
    return summarized_answer

Add pdf data and ask questions

In [35]:
def main():
    pdf_path = "C:/Users/gurme/Desktop/PDFs/ChessGame.pdf"
    pdf_text = extract_text_from_pdf(pdf_path)
    
    while True:
        question = input("Ask a question about the PDF (or type 'quit' to exit): ")
        if question.lower() == 'quit':
            break
        answer = get_answer(question, pdf_text)
        print("\nAnswer:", answer)

if __name__ == "__main__":
    main()


Answer: The exact same position does not have to happen three times in a row.

Answer: It can become a Queen, Rook, Bishop, or Knight. THE QUEEN = 9 =  
 
 
 
Each side has 1 King.

Answer: It can become a Queen, Rook, Bishop, or Knight. •You cannot castle if there are pieces between the King and the Rook.


In [39]:
pip install llama-cpp-python --prefer-binary --extra-index-url=https://llama-cpp-python.s3-us-west-2.amazonaws.com/cpu/

Looking in indexes: https://pypi.org/simple, https://llama-cpp-python.s3-us-west-2.amazonaws.com/cpu/
  Using cached llama_cpp_python-0.2.84.tar.gz (49.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for diskcache>=5.6.1 from https://files.pythonhosted.org/packages/3f/27/4570e78fc0bf5ea0ca45eb1de3818a23787af9b390c0b0a0033a1b8236f9/diskcache-5.6.3-py3-none-any.whl.metadata
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Failed to build llama-cpp-python
Note: you may need to restart the kernel to 

  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.9.9 using CMake 3.30.1 (wheel)
      *** Configuring CMake...
      2024-07-30 20:40:07,567 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\gurme\AppData\Local\Temp\tmp_o8dwnbe\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error or

Retrieving notices: ...working... done
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\gurme\anaconda3

  added / updated specs:
    - llama-cpp-python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-cloud-auth-0.5.0  |     pyhd8ed1ab_1          25 KB  conda-forge
    annotated-types-0.7.0      |     pyhd8ed1ab_0          18 KB  conda-forge
    ca-certificates-2024.7.4   |       h56e8100_0         151 KB  conda-forge
    certifi-2024.7.4           |     pyhd8ed1ab_0         156 KB  conda-forge
    conda-23.7.4               |  py311h1ea47a8_0         1.3 MB  conda-forge
    cuda-cudart-1



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.7.1


